[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/07-lcel.ipynb)

#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

In [35]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  docarray==0.40.0

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [36]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-lcel-openai"

---

## Traditional Chains vs LCEL

In this section we're going to dive into a basic example using the traditional method for building chains before jumping into LCEL. We will build a pipeline where the user must input a specific topic, and then the LLM will look and return a report on the specified topic. Generating a _research report_ for the user.

### Traditional LLMChain

The `LLMChain` is the simplest chain originally introduced in LangChain. This chain takes a prompt, feeds it into an LLM, and _optionally_ adds an output parsing step before returning the result.

Let's see how we construct this using the traditional method, for this we need:

* `prompt` — a `PromptTemplate` that will be used to generate the prompt for the LLM.
* `llm` — the LLM we will be using to generate the output.
* `output_parser` — an optional output parser that will be used to parse the structured output of the LLM.

In [1]:
from langchain import PromptTemplate

prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

For the LLM, we'll start by initializing our connection to the OpenAI API. We do need an OpenAI API key, which you can get from the [OpenAI platform](https://platform.openai.com/api-keys).

We will use the `gpt-4o-mini` model with a `temperature` of `0.0`:

In [2]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
)

In [3]:
llm_out = llm.invoke("Hello there")
llm_out

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-021dbdb0-8be2-4f36-ac9d-f669a8d39d70-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Then we define our output parser, this will be used to parse the output of the LLM. In this case, we will use the `StrOutputParser` which will parse the `AIMessage` output from our LLM into a single string.

In [4]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [5]:
out = output_parser.invoke(llm_out)
out

'Hello! How can I assist you today?'

Through the `LLMChain` class we can place each of our components into a linear `chain`.

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

/var/folders/7l/93h0p1b103v6bmb2440lc3740000gn/T/ipykernel_61505/2960353250.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)


Note that the `LLMChain` _was_ deprecated in LangChain `0.1.17`, the expected way of constructing these chains today is through LCEL, which we'll cover in a moment.

We can `invoke` our `chain`, providing a `topic` that we'd like to be researched.

In [7]:
result = chain.invoke("retrieval augmented generation")
result

{'topic': 'retrieval augmented generation',
 'text': '### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.\n\n#### Concept Overview\nRAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:\n\n1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It uses techniques such as vector embeddings and similarity search to identify the most pertinent data.\n\n2. **Generation Component**: After retrieving the relevant information, the generative model (often based on arch

We can view a formatted version of this output using the `Markdown` display:

In [8]:
from IPython.display import display, Markdown

display(Markdown(result["text"]))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.

#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:

1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It uses techniques such as vector embeddings and similarity search to identify the most pertinent data.

2. **Generation Component**: After retrieving the relevant information, the generative model (often based on architectures like Transformers) synthesizes a coherent and contextually appropriate response. This model leverages the retrieved data to enhance its output, ensuring that the generated text is not only fluent but also factually grounded.

#### Advantages
- **Enhanced Knowledge Access**: RAG allows models to tap into vast external knowledge bases, making them more informed and capable of providing accurate information.
- **Improved Contextual Relevance**: By retrieving specific documents related to a query, the generated responses are more contextually relevant and tailored to user needs.
- **Dynamic Learning**: RAG systems can be updated with new information without retraining the entire model, allowing for continuous improvement and adaptation to new data.

#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing precise answers to user queries by retrieving relevant documents and generating responses based on them.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with up-to-date information and contextually aware responses.
- **Content Creation**: Assisting in generating articles, summaries, or reports by pulling in relevant data from various sources.

#### Challenges
Despite its advantages, RAG also faces several challenges:
- **Quality of Retrieved Information**: The effectiveness of the generation depends heavily on the quality and relevance of the retrieved documents.
- **Complexity of Integration**: Combining retrieval and generation components can introduce complexity in system design and implementation.
- **Computational Resources**: RAG systems may require significant computational resources, especially when dealing with large corpora for retrieval.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing. By effectively combining retrieval and generation, RAG systems can produce more accurate, relevant, and context-aware outputs. As research and development in this area continue to evolve, RAG is poised to play a crucial role in various applications, enhancing the capabilities of AI-driven systems in understanding and generating human language. 

#### Future Directions
Future research may focus on improving the efficiency of retrieval mechanisms, enhancing the integration of diverse data sources, and developing methods to ensure the reliability and trustworthiness of the information retrieved. Additionally, exploring the ethical implications and biases in RAG systems will be essential for responsible AI deployment.

That is a simple `LLMChain` using the traditional LangChain method. Now let's move onto LCEL.

## LangChain Expression Language (LCEL)

**L**ang**C**hain **E**xpression **L**anguage (LCEL) is the recommended approach to building chains in LangChain. Having superceeded the traditional methods with `LLMChain`, etc. LCEL gives us a more flexible system for building chains. The pipe operator `|` is used by LCEL to _chain_ together components. Let's see how we'd construct an `LLMChain` using LCEL.

In [45]:
lcel_chain = prompt | llm | output_parser

We can `invoke` this chain in the same way as we did before:

In [46]:
result = lcel_chain.invoke("retrieval augmented generation")
result

"### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an advanced approach in natural language processing (NLP) that combines the strengths of information retrieval and generative models. This technique enhances the capabilities of language models by allowing them to access external knowledge bases or documents during the generation process, thereby improving the relevance and accuracy of the generated content.\n\n#### Key Components\n1. **Retrieval Mechanism**: RAG employs a retrieval system to fetch relevant documents or pieces of information from a large corpus based on the input query. This is typically done using techniques such as vector embeddings or traditional keyword matching.\n\n2. **Generative Model**: After retrieving relevant information, a generative model (often based on transformer architectures like BERT or GPT) processes this information along with the original query to produce coherent and contextually appr

The output format is slightly different, but the underlying functionality and content being output is the same. As before, we can view a formatted version of this output using the `Markdown` display:

In [47]:
display(Markdown(result))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an advanced approach in natural language processing (NLP) that combines the strengths of information retrieval and generative models. This technique enhances the capabilities of language models by allowing them to access external knowledge bases or documents during the generation process, thereby improving the relevance and accuracy of the generated content.

#### Key Components
1. **Retrieval Mechanism**: RAG employs a retrieval system to fetch relevant documents or pieces of information from a large corpus based on the input query. This is typically done using techniques such as vector embeddings or traditional keyword matching.

2. **Generative Model**: After retrieving relevant information, a generative model (often based on transformer architectures like BERT or GPT) processes this information along with the original query to produce coherent and contextually appropriate responses.

3. **Integration**: The integration of retrieval and generation can occur in various ways, such as:
   - **End-to-End Training**: The retrieval and generation components are trained together, allowing the model to learn how to select the most relevant information for generating responses.
   - **Pipeline Approach**: The retrieval and generation processes are handled separately, where the retrieval system first fetches relevant documents, and the generative model then uses these documents to create responses.

#### Advantages
- **Enhanced Knowledge Access**: RAG allows models to leverage vast external knowledge, making them more informed and capable of answering a wider range of queries.
- **Improved Accuracy**: By grounding responses in retrieved documents, RAG can reduce hallucinations (the generation of incorrect or nonsensical information) that are common in standalone generative models.
- **Dynamic Updates**: The retrieval component can be updated independently of the generative model, allowing the system to incorporate new information without retraining the entire model.

#### Applications
- **Question Answering**: RAG is particularly effective in systems designed for question answering, where users seek specific information that may not be contained within the model's training data.
- **Chatbots and Virtual Assistants**: By providing contextually relevant information, RAG can enhance the conversational abilities of chatbots, making interactions more informative and engaging.
- **Content Creation**: RAG can assist in generating articles, summaries, or reports by retrieving relevant data and synthesizing it into coherent narratives.

#### Challenges
- **Retrieval Quality**: The effectiveness of RAG heavily depends on the quality of the retrieval system. Poor retrieval can lead to irrelevant or misleading information being used in the generation process.
- **Computational Complexity**: The dual nature of retrieval and generation can increase the computational resources required, making it less efficient than simpler models in some scenarios.
- **Integration Complexity**: Balancing the interaction between the retrieval and generation components can be complex, requiring careful tuning and optimization.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of NLP, combining the strengths of information retrieval and generative modeling to create more accurate and contextually relevant outputs. As research and development in this area continue, RAG is likely to play a crucial role in the evolution of intelligent systems capable of understanding and generating human-like text.

### How Does the Pipe Operator Work?

Before moving onto other LCEL features, let's take a moment to understand what the pipe operator `|` is doing and _how_ it works.

Functionality wise, the pipe tells you that whatever the _left_ side outputs will be fed as input into the _right_ side. In the example of `prompt | llm | output_parser`, we see that `prompt` feeds into `llm` feeds into `output_parser`.

The pipe operator is a way of chaining together components, and is a way of saying that whatever the _left_ side outputs will be fed as input into the _right_ side.

Let's make a basic class named `Runnable` that will transform our a provided function into a _runnable_ class that we will then use with the pipe `|` operator.

In [48]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)

With the `Runnable` class, we will be able wrap a function into the class, allowing us to then chain together multiple of these _runnable_ functions using the `__or__` method.

First, let's create a few functions that we'll chain together:

In [49]:
def add_five(x):
    return x+5

def sub_five(x):
    return x-5

def mul_five(x):
    return x*5

Now we wrap our functions with the `Runnable`:

In [50]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_five)
mul_five_runnable = Runnable(mul_five)

Finally, we can chain these together using the `__or__` method from the `Runnable` class:

In [51]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

chain.invoke(3)

15

So we can see that we're able to chain together our functions using `__or__`. The pipe `|` operator is simply a shortcut for the `__or__` method, so we can create the exact same chain like so:

In [52]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

chain.invoke(3)

15

## LCEL `RunnableLambda`

The `RunnableLambda` class is LangChain's built-in method for constructing a _runnable_ object from a function. That is, it does the same thing as the custom `Runnable` class we created earlier. Let's try it out with the same functions as before.

In [53]:
from langchain_core.runnables import RunnableLambda

add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_five)
mul_five_runnable = RunnableLambda(mul_five)

We chain these together again with the pipe `|` operator:

In [54]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

And call them using the `invoke` method:

In [55]:
chain.invoke(3)

15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [56]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [57]:
chain = prompt | llm | output_parser

In [58]:
result = chain.invoke("AI")
display(Markdown(result))

### Report on Artificial Intelligence (AI)

#### Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. AI has rapidly evolved over the past few decades, becoming an integral part of various industries and daily life.

#### Historical Context
The concept of AI dates back to the mid-20th century, with pioneers like Alan Turing and John McCarthy laying the groundwork. The term "artificial intelligence" was coined in 1956 during the Dartmouth Conference, which is considered the birth of AI as a field. Early AI research focused on problem-solving and symbolic methods, but progress was slow due to limited computational power and data.

#### Current Trends
1. **Machine Learning (ML)**: A subset of AI that enables systems to learn from data and improve over time without explicit programming. Deep learning, a branch of ML, uses neural networks to analyze vast amounts of data, leading to breakthroughs in image and speech recognition.

2. **Natural Language Processing (NLP)**: This technology allows machines to understand and respond to human language. Applications include chatbots, virtual assistants, and language translation services.

3. **Robotics**: AI is increasingly integrated into robotics, enabling machines to perform complex tasks in manufacturing, healthcare, and logistics. Autonomous vehicles are a prominent example of AI in robotics.

4. **Ethics and Regulation**: As AI technologies advance, ethical considerations and regulatory frameworks are becoming critical. Issues such as bias in algorithms, data privacy, and the impact of automation on jobs are at the forefront of discussions among policymakers and technologists.

#### Applications
AI is transforming various sectors, including:

- **Healthcare**: AI algorithms assist in diagnosing diseases, personalizing treatment plans, and managing patient data.
- **Finance**: AI is used for fraud detection, algorithmic trading, and customer service through chatbots.
- **Retail**: Personalized shopping experiences, inventory management, and supply chain optimization are enhanced by AI.
- **Entertainment**: Streaming services use AI to recommend content based on user preferences.

#### Challenges
Despite its potential, AI faces several challenges:

- **Data Privacy**: The collection and use of personal data raise concerns about privacy and security.
- **Bias and Fairness**: AI systems can perpetuate existing biases if trained on biased data, leading to unfair outcomes.
- **Job Displacement**: Automation may lead to job losses in certain sectors, necessitating workforce retraining and adaptation.

#### Conclusion
Artificial Intelligence is a rapidly evolving field with the potential to revolutionize industries and improve quality of life. However, it also presents significant challenges that require careful consideration and proactive management. As AI continues to advance, collaboration among technologists, ethicists, and policymakers will be essential to harness its benefits while mitigating risks.

#### Recommendations
- **Invest in Education**: Promote AI literacy and training programs to prepare the workforce for an AI-driven economy.
- **Establish Ethical Guidelines**: Develop frameworks to ensure the responsible use of AI technologies.
- **Encourage Collaboration**: Foster partnerships between academia, industry, and government to address challenges and drive innovation.

---

This report provides a brief overview of AI, its current trends, applications, challenges, and recommendations for future development.

Here we are making two functions, `extract_fact` to pull out the main content of our text and `replace_word` that will replace AI with Skynet!

In [9]:
# \n 表示换行符
# \n\n 表示空行
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

old_word = "AI"
new_word = "skynet"

def replace_word(x):
    return x.replace(old_word, new_word)

In [13]:
x = """你好，
我的名字是lxy

我想开发AI agent, 

你想开发吗？
"""
print(extract_fact(x))
print(replace_word(x))

"\n".join(x.split("\n\n"))

我想开发AI agent, 
你想开发吗？

你好，
我的名字是lxy

我想开发skynet agent, 

你想开发吗？



'你好，\n我的名字是lxy\n我想开发AI agent, \n你想开发吗？\n'

Lets wrap these functions and see what the output is!

In [60]:
extract_fact_runnable = RunnableLambda(extract_fact)
replace_word_runnable = RunnableLambda(replace_word)

In [61]:
chain = prompt | llm | output_parser | extract_fact_runnable | replace_word_runnable

In [62]:
result = chain.invoke("retrieval augmented generation")
display(Markdown(result))

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.
#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:
1. **Retrieval Component**: This part of the system searches a large corpus of documents or knowledge bases to find relevant information based on a given query. It employs techniques such as vector embeddings and similarity search to identify the most pertinent documents.
2. **Generation Component**: Once relevant documents are retrieved, the generative model (often based on architectures like Transformers) synthesizes a coherent response by incorporating the retrieved information. This allows the model to produce more informed and contextually appropriate outputs.
#### Advantages
- **Enhanced Knowledge Access**: RAG allows models to leverage vast external datasets, which can significantly improve the quality of responses, especially in domains requiring up-to-date or specialized knowledge.
- **Reduced Hallucination**: Traditional generative models sometimes produce inaccurate or fabricated information (a phenomenon known as "hallucination"). By grounding responses in retrieved documents, RAG can mitigate this issue.
- **Dynamic Adaptability**: The retrieval component can be updated independently of the generative model, allowing the system to adapt to new information without retraining the entire model.
#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing accurate answers to user queries by retrieving relevant documents and generating responses based on that information.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with the ability to pull in real-time data and provide contextually relevant answers.
- **Content Creation**: Assisting in generating articles, reports, or summaries by retrieving and synthesizing information from multiple sources.
#### Challenges
Despite its advantages, RAG faces several challenges:
- **Complexity**: The integration of retrieval and generation components can complicate the system architecture and increase computational requirements.
- **Quality of Retrieved Information**: The effectiveness of RAG heavily depends on the quality and relevance of the retrieved documents. Poor retrieval can lead to suboptimal generation.
- **Latency**: The retrieval process can introduce delays, which may affect user experience in real-time applications.
#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing, combining the strengths of retrieval and generation to produce more accurate and contextually relevant outputs. As research and development in this area continue, RAG is poised to play a crucial role in various applications, enhancing the capabilities of skynet systems in understanding and generating human-like text. Future work will likely focus on improving retrieval efficiency, enhancing the quality of generated content, and addressing the challenges associated with system complexity and latency.

Those are our `RunnableLambda` functions. It's worth noting that all inputs to these functions are expected to be a SINGLE arguments. If you have a function that accepts multiple arguments, you can input a dictionary with keys, then unpack them inside the function.

RunnableLambda 是 LangChain 中用来包装普通 Python 函数的类，让它们可以在 LCEL 管道中使用。 \n
它能包装的函数只能有一个参数。如果函数要传多个参数，可以使用字典传参，在函数内部解包

In [ ]:
# 原始函数（多参数）
def calculate_total(price, tax_rate, discount):
    total = price * (1 + tax_rate) * (1 - discount)
    return round(total, 2)

# 普通调用
result = calculate_total(100, 0.1, 0.2)  # 88.0
# 内部解包
def calculate_total_unpack(data):
    # 解包字典
    return calculate_total(**data)

calc_runnable = RunnableLambda(calculate_total_unpack)

# 使用
result = calc_runnable.invoke({
    'price': 100,
    'tax_rate': 0.1,
    'discount': 0.2
})
print(result)  # 88.0

## LCEL `RunnableParallel` and `RunnablePassthrough`

LCEL provides us with various `Runnable` classes that allow us to control the flow of data and execution order through our chains. Two of these are `RunnableParallel` and `RunnablePassthrough`.

* `RunnableParallel` — allows us to run multiple `Runnable` instances in parallel. Acting almost as a Y-fork in the chain.

* `RunnablePassthrough` — allows us to pass through a variable to the next `Runnable` without modification.

To see these runnables in action, we will create two data sources, each source provides specific information but to answer the question we will need both to fed to the LLM.

In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

# 创建嵌入模型： OpenAIEmbeddings()用来将文本转换为向量表示
embedding = OpenAIEmbeddings()

# 创建两个向量数据库
vecstore_a = DocArrayInMemorySearch.from_texts(
    [
        "half the info is here",
        "DeepSeek-V3 was released in December 2024"
    ],
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    [
        "the other half of the info is here",
        "the DeepSeek-V3 LLM is a mixture of experts model with 671B parameters"
    ],
    embedding=embedding
)

/var/folders/7l/93h0p1b103v6bmb2440lc3740000gn/T/ipykernel_61505/3699332964.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/Users/liuxinyi/lxy_learning/langchain_learning/langchain-course/.venv/lib/python3.12/site-packages/docarray/helper.py:224: SyntaxWarning: invalid escape sequence '\*'
  """
/Users/liuxinyi/lxy_learning/langchain_learning/langchain-course/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Here you can see the prompt does have three inputs, two for context and one for the question itself.

In [ ]:
prompt_str = """Using the context provided, answer the user's question.
Context:
{context_a}
{context_b}
"""

In [65]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt_str),
    HumanMessagePromptTemplate.from_template("{question}")
])

Here we are wrapping our vector stores as retrievers so they can be fitted into one big retrieval variable to be used by the prompt.

RunnablePassthrough() 就像一个"原样传递"的管道，它的作用是：接收到什么，就输出什么，不做任何处理。

In [66]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# 将向量存储（VectorStore） 转换为 检索器（Retriever）
# 向量存储主要用于存储和管理向量数据
# 检索器专门用于根据查询检索相关文档
retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, 
        "context_b": retriever_b, 
        "question": RunnablePassthrough()
    }
)

The chain we'll be constructing will look something like this:

![](https://github.com/aurelio-labs/langchain-course/blob/main/assets/lcel-flow.png?raw=1)

In [67]:
chain = retrieval | prompt | llm | output_parser

We `invoke` it as usual.

In [68]:
result = chain.invoke(
    "what architecture does the model DeepSeek released in december use?"
)
result

'The DeepSeek-V3 model, released in December 2024, uses a mixture of experts architecture with 671 billion parameters.'

With that we've seen how we can use `RunnableParallel` and `RunnablePassthrough` to control the flow of data and execution order through our chains.

---

什么是 Runnable？ \\\
Runnable 是 LangChain 中的一个核心概念，它是一个标准化的接口，任何实现了这个接口的对象都可以：


被调用：有 invoke() 方法


被链接：支持 | 操作符


被组合：可以与其他 Runnable 连接


In [ ]:
# 第一种写法
# 但输入数据格式已经完全匹配prompt期望的格式时，可以直接使用prompt
chain = prompt | llm | output_parser

# prompt定义 期望的格式
prompt = PromptTemplate(
    input_variables=["topic"],
    template="give me a small report about {topic}"
)

# 直接调用，输入格式匹配
chain.invoke({"topic": "AI"})  # ✅ 直接匹配
# 或者
chain.invoke("AI")  # ✅ 如果只有一个变量，可能支持直接传字符串

In [ ]:
# 假设你的输入是这样的复杂格式
input_data = {
    "user_query": "What is RAG?",
    "retrieved_docs": ["doc1", "doc2"],
    "metadata": {...}
}

# 但prompt期望的是这样的格式
prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template="Based on context: {context}, answer: {query}"
)

# 所以需要先转换
pipeline = (
    {
        "query": lambda x: x["user_query"],      # 从user_query提取到query
        "context": lambda x: x["retrieved_docs"] # 从retrieved_docs提取到context
    }
    | prompt_template  # 现在格式匹配了
    | llm
)

result = pipeline.invoke(input_data)